## Entregable 1: 
### Preprocesamiento de datos multivariados o de una serie temporal
#### Manolo Ramírez Pintor - A01706155

1. #### Realiza un análisis exploratorio de las variables:

*   * **Seleccion y limpieza de datos**

In [1]:
# Importamos las librerías necesarias para trabajar:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Leemmos los datasets y quitamos columnas que no necesitamos
us2022 = pd.read_csv('datasets/us2022q2a.csv')
usfirms = pd.read_csv('datasets/usfirms2022.csv')
us_merged = us2022.merge(usfirms, left_on='firm', right_on='Ticker')

In [3]:
# Selecciono únicamente los sectores Retail Trade y Wholesale Trade
us_merged = us_merged.loc[us_merged['Sector NAICS\nlevel 1'].isin(['Retail Trade', 'Wholesale Trade'])]
# us_merged

In [4]:
us_merged['R'] = np.log(us_merged.groupby(['firm'])['adjprice'].shift(-1)) - np.log(us_merged.groupby(['firm'])['adjprice'].shift(3))

In [5]:
df_masked = us_merged[us_merged['q'] == '2022q1']

In [6]:
df_masked = df_masked.drop(['extraincome', 'shortdebt', 'longdebt', 'stockholderequity', 
                'Ticker', 'Name', 'Class', 'Exchange / Src', 'Sector\nEconomatica', 
                'Sector NAICS\nlast available', 'partind', 'N', 'Country\nof Origin', 
                'Type of Asset', 'fiscalmonth', 'year', 'cto'], axis=1)
df_masked

,firm,q,revenue,cogs,sgae,otheropexp,finexp,incometax,totalassets,totalliabilities,adjprice,originalprice,sharesoutstanding,Sector NAICS\nlevel 1,R
718,AAP,2022q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.158154,206.96,61097.579,Retail Trade,-0.144831
1168,ABC,2022q1,57719448.0,55484368.0,1378528.0,76395.0,52916.0,172944.0,57308153.0,56402088.0,153.761909,154.71,209137.429,Wholesale Trade,0.225304
1435,ABG,2022q1,3911800.0,3119800.0,473900.0,-2700.0,40200.0,76000.0,7860100.0,5677600.0,160.200000,160.20,23187.817,Retail Trade,-0.011916
3142,ACI,2022q1,17383500.0,12399200.0,4321700.0,-1700.0,108000.0,148700.0,28123000.0,25098400.0,32.976197,33.25,483118.147,Retail Trade,0.323732
5932,AE,2022q1,774245.0,0.0,766097.0,0.0,90.0,1968.0,470117.0,304596.0,38.241532,38.49,4367.866,Wholesale Trade,0.181133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315984,WSM,2022q1,2501029.0,1375792.0,600665.0,0.0,-89.0,121720.0,4625620.0,2961413.0,143.394496,145.00,71558.874,Retail Trade,-0.346125
316074,WSO,2022q1,1523570.0,1073212.0,283354.0,-4045.0,558.0,35601.0,3473889.0,1393174.0,299.661871,304.64,38772.956,Wholesale Trade,-0.154603
316344,WSTG,2022q1,71319.0,59338.0,8606.0,0.0,10.0,796.0,188770.0,134861.0,34.197852,34.55,4450.062,Wholesale Trade,0.304832
322284,ZEUS,2022q1,696333.0,555107.0,88104.0,0.0,1998.0,13816.0,1043703.0,580917.0,38.364750,38.46,11123.700,Wholesale Trade,-0.124839


In [7]:
# Obtenemos calculos de variables en base a las fórmulas que hemos visto anteriormente

# Get the Ebit and store it on 'Ebit' var:
df_masked['EBIT'] = df_masked['revenue'] - df_masked['cogs'] - df_masked['sgae'] - df_masked['otheropexp']
# Get the Net Income and store it on 'Net income' var:
df_masked['NetIncome'] = df_masked['EBIT'] - df_masked['incometax'] - df_masked['finexp']
# Get the EPS and store it on 'EPS' var:
df_masked['EPS'] = df_masked['NetIncome'] / df_masked['sharesoutstanding']

# Get Firm sizes and store them on a 'Book value' var:
df_masked['Book'] = df_masked['totalassets'] - df_masked['totalliabilities']
# Get Market values and store them on a 'Market value' var:
df_masked['Market'] = df_masked['originalprice'] * df_masked['sharesoutstanding']
# Get Firm sizes ratio (Book to Market ratio) values and store them on a 'Market value' var:
df_masked['BTMR'] = df_masked['Book'] / df_masked['Market']

# Get EPSP and store it on a 'EPSP' var:
df_masked['EPSP'] = df_masked['EPS'] / df_masked['originalprice']
# Get the Operating Profit Margin and store it on 'OPM' var:
df_masked['OPM'] = df_masked['EBIT'] / df_masked['revenue']

In [8]:
# We sort the values for Market Value
df_masked = df_masked.sort_values(by = 'Market')
# The index is gets resetted 
df_masked = df_masked.reset_index()
df_masked = df_masked.drop(['index'], axis = 1)


# We make a new column for small and medium values of our sorted and masked data
df_masked["small"] = df_masked.index <= len(df_masked) / 3
df_masked["small"] = df_masked["small"].astype(int)

df_masked["medium"] = (df_masked.index <= (2 * len (df_masked)/3)) & (df_masked.index > (len(df_masked) / 3))
df_masked["medium"] = df_masked["medium"].astype(int)

In [10]:
# With a dataset copy, we make a drop of the null values.
# A filter or remasking could work but it's the exact same thing at the end.
df_process = df_masked[['R', 'EPSP', 'OPM', 'BTMR', 'small', 'medium']]
df_process = df_process.dropna()

x = df_process[['EPSP', 'OPM', 'BTMR', 'small', 'medium']].to_numpy
y = df_process['R'].to_numpy


-    - Calcula matriz de varianza y covarianza, así como matriz de correlación de las variables independientes y la dependiente. Explicar qué es la varianza, covarianza y correlación. Interpreta la matriz de correlación. Tiene que utilizar álgebra matricial y corroborar resultados con funciones de Python.



In [19]:
# We calculate que covariance matrix using a manual function:
# Refefence: https://docs.google.com/viewer?a=v&pid=sites&srcid=ZWdhZGVyemMubmV0fGZ6MzAzMHxneDozOTI1ODBmMGU2YTMzZGM1 last page and formula.
def covar_x(x):
    N = len(x)
    one = np.ones((N,1))
    X_1 = x.T @ x
    X_2 = x.T @ one
    X_3 = X_2.T
    varcov = (X_1 - (1/N) * (X_2 @ X_3)) / (N - 1)
    return varcov

covar_x(x)

,EPSP,OPM,BTMR,small,medium
EPSP,0.010281,0.010747,-0.002510,-0.011957,0.006929
OPM,0.010747,0.038002,-0.000704,-0.021568,0.005114
BTMR,-0.002510,-0.000704,0.183353,0.073453,0.005851
small,-0.011957,-0.021568,0.073453,0.219000,-0.110980
medium,0.006929,0.005114,0.005851,-0.110980,0.226715


In [16]:
# With pandas, we will check if the manual correlation function was right
x.cov()

,EPSP,OPM,BTMR,small,medium
EPSP,0.010281,0.010747,-0.002510,-0.011957,0.006929
OPM,0.010747,0.038002,-0.000704,-0.021568,0.005114
BTMR,-0.002510,-0.000704,0.183353,0.073453,0.005851
small,-0.011957,-0.021568,0.073453,0.219000,-0.110980
medium,0.006929,0.005114,0.005851,-0.110980,0.226715


In [21]:
# Now we get the variance from the matrix that we got.
x.var()

EPSP      0.010281
OPM       0.038002
BTMR      0.183353
small     0.219000
medium    0.226715
dtype: float64

In [20]:
# Con pandas vamos a ver si está bien la función de covarianza manual
x.corr()

,EPSP,OPM,BTMR,small,medium
EPSP,1.000000,0.543709,-0.057806,-0.251986,0.143513
OPM,0.543709,1.000000,-0.008431,-0.236421,0.055093
BTMR,-0.057806,-0.008431,1.000000,0.366559,0.028695
small,-0.251986,-0.236421,0.366559,1.000000,-0.498059
medium,0.143513,0.055093,0.028695,-0.498059,1.000000


- 
   - Corre pruebas estadísticas para detectar outliers y leverage points. Tiene que utilizar álgebra matricial para las pruebas y explicar claramente cómo funcionan las pruebas. Puede utilizar funciones de Python para corroborar resultados.

2. #### Hace un análisis de multicolinealidad explicando la prueba e implicaciones en el modelo:

3. #### Propone e implementa soluciones a los problemas de los puntos anteriores para que el modelo sea el más adecuado.

4. #### Estima e interpreta un modelo de regresión múltiple después de atender los problemas anteriores. Tiene que utilizar álgebra matricial para estimar coeficientes y errores estándar del modelo de regresión, y utilizar funciones de Python para corroborar resultados.

In [ ]:
mod_x = pd.DataFrame()
mod_x['1'] = 1
mod_x = x[['1', 'isSmall', 'isMedium', 'EPSP', 'OPM', 'Book_to_Market_ratio']]
mod_x = mod_x.to_numpy()
x_trans = np.transpose(mod_x)

Primer valor de x son 1

x´ es x transpuesta

x^1 es x inversa

(x´x)^-1x´y